In [1]:
from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

In [2]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-07-19 15:36:52--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Résolution de raw.githubusercontent.com (raw.githubusercontent.com)… 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connexion à raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443… connecté.
requête HTTP transmise, en attente de la réponse… 200 OK
Taille : 1115394 (1,1M) [text/plain]
Enregistre : «input.txt.1»

input.txt.1         100%[===================>]   1,06M  --.-KB/s    ds 0,06s   

2023-07-19 15:36:52 (17,1 MB/s) - «input.txt.1» enregistré [1115394/1115394]



In [3]:
with open('input.txt',mode='r',encoding='utf-8') as f:
    text = f.read()

In [4]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
itos = { k:v for k,v in enumerate(chars)}
stoi = { v:k for k,v in enumerate(chars)}
encoder = lambda string : [stoi[s] for s in string]
decoder = lambda lst : ''.join([itos[i] for i in lst])
print(text[:10],decoder(encoder(text[:10])))


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
First Citi First Citi


In [5]:
data = torch.tensor(encoder(text)).cuda()

In [6]:
train_data, val_data = data[:int(0.9*len(data))], data[int(0.9*len(data)):]

In [7]:
def get_batch(dataset,block_size,batch_size):
    start_idx = np.random.choice(len(dataset)-block_size,size=batch_size,replace=False)
    inp = [dataset[i:i+block_size] for i in start_idx]
    target = [dataset[i+1:i+block_size+1] for i in start_idx]
    return torch.stack(inp), torch.stack(target)

In [8]:
class BiGram(nn.Module):
    def __init__(self,n_vocab):
        super().__init__()
        self.embedding = nn.Embedding(n_vocab,n_vocab)
    
    def forward(self,inp):
        
        logits = self.embedding(inp)
        return logits
    def generate(self,inp,sequence_length):
        out = [inp]
        cur_tokens = inp
        for _ in range(sequence_length):
            logits = self(cur_tokens)[:,-1]
            probabilities = F.softmax(logits,dim=-1)
            cur_tokens = torch.multinomial(probabilities,num_samples=1)
            out.append(cur_tokens)
            
        return torch.stack(out).transpose(0,1)[...,0]


In [10]:
model = BiGram(vocab_size).cuda()
opt = torch.optim.Adam(model.parameters(),lr=1e-3)
batch_size = 1024
itr_per_epoch = int(len(train_data)/10/batch_size)
val_itr_per_epoch = int(len(val_data)/10/batch_size)
x,y = get_batch(train_data,1,8)
print(decoder(model.generate(x,100)[0].detach().cpu().numpy()))

oqU:MLmos,aeNk $iVIn3zUzne$EtCXvtJLP.SQ!;$EtCuhFCMZQFqi?Vfj
-CZfFCCzqiUKP.:,EDc!QjBR$DbjeAHSbqU'geZPo


In [11]:
pbar = tqdm(range(100))
losses, val_losses = [], []
for e in pbar:
    for i in range(itr_per_epoch):
        x,y  = get_batch(train_data,1,batch_size)
        opt.zero_grad()
        logits = model(x)[:,0]
        loss = F.cross_entropy(logits,y.view(-1))
        loss.backward()
        losses.append(loss.item())
        opt.step()
    model.eval()
    with torch.no_grad():
        for i in range(val_itr_per_epoch):
            x,y = get_batch(val_data,1,batch_size)
            logits = model(x)[:,0]
            loss = F.cross_entropy(logits,y.view(-1))
            val_losses.append(loss.item())
    model.train()
    pbar.set_description('Loss : {:.3e}, Val : {:.3e}'.format(
        np.mean(losses[-itr_per_epoch:]),np.mean(val_losses[-val_itr_per_epoch:])))

Loss : 2.459e+00, Val : 2.471e+00: 100%|██████████| 100/100 [06:27<00:00,  3.87s/it]


In [12]:
x,y = get_batch(train_data,1,1)
print(decoder(model.generate(x,1000)[0].detach().cpu().numpy()))

urommis, sodise gon
Bunot met:


K: ve CLIINGS:
Arayocy mfory e

I shieshelo? n ome imefoerar'soutse athesis avasiayo ismaieldrs s k henling th hores:
SI'de! thteale s, habl not y g torwo tes wim atad venoo ne s art touthand he h ancashtate t s ar.
I t mouplevooutere ancoweay ainfopeche g;
PS:
M: t t t dere we m douarcthengld m buntheeecha k harebby rse:
Wheorthe prrskllarkitisorsurrus, whod as lyou che thonownghry u his t ade at wo berph be! boot,
LOOLOx' RWhrspendetuY:
Wenetheacaco'ldis ckn:
ICLLENCERENI'd s'dyokig the rethore bsculldy

OMadur wind Riaklathedys
Wen oune wemouro wrver wake!
S: h, jurs
BUSowieshouge u r sticSpranody se in'ser tharifertarhave, sem tre Irtha.
An dst thee awf t nding bllis al:
SCE IV:
Who theif thtte agshe y?
Fsutin tthouifl, frea stisck ve o lle'g h s:
S:
indd adesise lule d n tmar
Tha y he ndsell!
BRERE cal---gh Is dwof, y, wamod boferst t ppeindave mell nnde t.
LAs icthe theas upit t retr

This IOFou f lime k, des:
I me wisthed cethere inas:
Fed
Bupene